# Cleaning data and the skies

## 📖 Background
Your are a data analyst at an environmental company. Your task is to evaluate ozone pollution across various regions.

You’ve obtained data from the U.S. Environmental Protection Agency (EPA), containing daily ozone measurements at monitoring stations across California. However, like many real-world datasets, it’s far from clean: there are missing values, inconsistent formats, potential duplicates, and outliers.

Before you can provide meaningful insights, you must clean and validate the data. Only then can you analyze it to uncover trends, identify high-risk regions, and assess where policy interventions are most urgently needed.

## 💾 The data
The data is a modified dataset from the U.S. Environmental Protection Agency ([EPA](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)). 

#### Ozone contains the daily air quality summary statistics by monitor for the state of California for 2024. Each row contains the date and the air quality metrics per collection method and site
- "Date" - the calendar date with which the air quality values are associated
- "Source" - the data source: EPA's Air Quality System (AQS), or Airnow reports
- "Site ID" - the id for the air monitoring site
- "POC" - the id number for the monitor
- "Daily Max 8-hour Ozone Concentration" - the highest 8-hour value of the day for ozone concentration
- "Units" - parts per million by volume (ppm)
- "Daily AQI Value" - the highest air quality index value for the day, telling how clean or polluted the air is (a value of 50 represents good air quality, while a value above 300 is hazardous)
- "Local Site Name" - name of the monitoring site
- "Daily Obs Count" - number of observations reported in that day
- "Percent Complete" - indicates whether all expected samples were collected
- "Method Code" - identifier for the collection method
- "CBSA Code" - identifier for the core base statistical area (CBSA)
- "CBSA Name" - name of the core base statistical area
- "State FIPS Code" - identifier for the state
- "State" - name of the state
- "County FIPS Code" - identifer for the county
- "County" - name of the county
- "Site Latitude" - latitude coordinates of the site
- "Site Longitude" - longitude coordinates of the side

In [43]:
import pandas as pd
from pathlib import Path
ozone = pd.read_csv(Path.cwd().parent / 'data/ozone.csv')
ozone.head()

,Date,Source,Site ID,POC,Daily Max 8-hour Ozone Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,Method Code,CBSA Code,CBSA Name,County FIPS Code,County,Site Latitude,Site Longitude
0,/2024,AQS,60010007,1,0.031,ppm,29.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
1,01/02/2024,AQS,60010007,1,0.037,ppm,34.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
2,/2024,AQS,60010007,1,NaN,ppm,30.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
3,January 04/2024,AQS,60010007,1,0.026,ppm,24.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
4,January 05/2024,AQS,60010007,1,0.027,ppm,25.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217


## 💪 Competition challenge

Create a report that covers the following:
1. Your EDA and data cleaning process.
2. How does daily maximum 8-hour ozone concentration vary over time and regions?
3. Are there any areas that consistently show high ozone concentrations? Do different methods report different ozone levels?
4. Consider if urban activity (weekend vs. weekday) has any affect on ozone levels across different days.
5. Bonus: plot a geospatial heatmap showing any high ozone concentrations.

## 🧑‍⚖️ Judging criteria

| CATEGORY | WEIGHTING | DETAILS                                                              |
|:---------|:----------|:---------------------------------------------------------------------|
| **Recommendations** | 35%       | <ul><li>Clarity of recommendations - how clear and well presented the recommendation is.</li><li>Quality of recommendations - are appropriate analytical techniques used & are the conclusions valid?</li><li>Number of relevant insights found for the target audience.</li></ul>       |
| **Storytelling**  | 35%       | <ul><li>How well the data and insights are connected to the recommendation.</li><li>How the narrative and whole report connects together.</li><li>Balancing making the report in-depth enough but also concise.</li></ul> |
| **Visualizations** | 20% | <ul><li>Appropriateness of visualization used.</li><li>Clarity of insight from visualization.</li></ul> |
| **Votes** | 10% | <ul><li>Up voting - most upvoted entries get the most points.</li></ul> |

## ✅ Checklist before publishing into the competition
- Rename your workspace to make it descriptive of your work. N.B. you should leave the notebook name as notebook.ipynb.
- **Remove redundant cells** like the judging criteria, so the workbook is focused on your story.
- Make sure the workbook reads well and explains how you found your insights. 
- Try to include an **executive summary** of your recommendations at the beginning.
- Check that all the cells run without error

# Exploratory Data Analysis (EDA)

## Data Summary

Let's examine our ozone dataset to understand its structure, dimensions, and potential issues before performing any analysis.

In [41]:
ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54759 entries, 0 to 54758
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Date                                  54759 non-null  object 
 1   Source                                54759 non-null  object 
 2   Site ID                               54759 non-null  int64  
 3   POC                                   54759 non-null  int64  
 4   Daily Max 8-hour Ozone Concentration  52021 non-null  float64
 5   Units                                 54759 non-null  object 
 6   Daily AQI Value                       52021 non-null  float64
 7   Local Site Name                       54759 non-null  object 
 8   Daily Obs Count                       54759 non-null  int64  
 9   Percent Complete                      54759 non-null  float64
 10  Method Code                           48269 non-null  float64
 11  CBSA Code      

The dataset contains 54,759 rows and 16 columns, representing daily ozone measurements from various monitoring stations across California for 2024. Key observations:

- **Missing Values**: There are some missing values in the "Daily Max 8-hour Ozone Concentration" column and the same amount in "Daily AQI Value", indicating these metrics are directly related.
  
- **Measurement Methods**: "Method Code" also has missing values, suggesting incomplete documentation of collection methodologies.

## Statistical Summary

Let's examine the key statistical characteristics of our dataset to understand the distribution of ozone measurements and other numerical variables. This will help identify potential outliers and understand the central tendencies of our data. It will also help us determine what values might be considered normal versus anomalous in our dataset.



In [12]:
ozone.describe()

,Site ID,POC,Daily Max 8-hour Ozone Concentration,Daily AQI Value,Daily Obs Count,Percent Complete,Method Code,CBSA Code,County FIPS Code,Site Latitude,Site Longitude
count,5.475900e+04,54759.000000,52021.000000,52021.000000,54759.000000,54759.000000,48269.000000,52351.000000,54759.000000,54759.000000,54759.000000
mean,6.059086e+07,1.000256,0.043519,45.280079,27.534688,99.340346,89.051669,36107.577697,58.918406,36.169550,-119.665107
std,3.001943e+05,0.015988,0.014610,25.899065,97.800446,3.658452,36.636778,9536.717562,30.012874,2.192841,2.022987
min,6.001001e+07,1.000000,0.000000,0.000000,1.000000,6.000000,47.000000,12540.000000,1.000000,32.578165,-124.179490
25%,6.031100e+07,1.000000,0.034000,31.000000,17.000000,100.000000,87.000000,31080.000000,31.000000,34.181977,-121.420838
50%,6.065500e+07,1.000000,0.041000,38.000000,17.000000,100.000000,87.000000,40140.000000,65.000000,36.209286,-119.828400
75%,6.081100e+07,1.000000,0.052000,48.000000,17.000000,100.000000,87.000000,41860.000000,81.000000,37.961578,-118.068500
max,6.113100e+07,2.000000,0.139000,236.000000,1000.000000,100.000000,199.000000,49700.000000,113.000000,41.726892,-114.602090


The statistical analysis reveals important characteristics of our key measurements:

- **Ozone Concentration**: 
  - Range: 0 to 0.1390 ppm
  - Mean: 0.043519 ppm
  - Median (50%): 0.0410 ppm
  - The min range (0 ppm) might be missing values or days with no ozone detected
  - The mean (0.0435 ppm) and median (0.0410 ppm) are very close, which suggests a nearly symmetric distribution

- **Daily AQI Value**:
  - Range: 0 to 236
  - Mean: 45.28
  - Median (50%): 38
  - Most values fall within acceptable air quality ranges, but some extreme values (236) indicate hazardous conditions

- **Daily Obs Count**:
  - Range: 1 to 1000
  - Mean: 27.5347
  - Median (50%): 17
  - The maximum daily observation count of 1000 is an outlier, suggesting a deeper investigation

- **Geographic Coverage**:
  - Sites spread across latitudes (32.58°N to 41.73°N) and longitudes (-124.18°E to -114.60°E)
  - This covers the entire state of California from the coast to inland regions

- **Data Completeness**:
  - Most measurements show 100% completeness (mean 99.34%)

This initial analysis suggests the data is generally reliable with some outliers that warrant further investigation, especially in the high AQI ranges and daily observation counts.

## Feature Selection

In [18]:
# First, let's see all column names
print("All columns in the dataset:")
print(ozone.columns)

All columns in the dataset:
Index(['Date', 'Source', 'Site ID', 'POC',
       'Daily Max 8-hour Ozone Concentration', 'Units', 'Daily AQI Value',
       'Local Site Name', 'Daily Obs Count', 'Percent Complete', 'Method Code',
       'CBSA Code', 'CBSA Name', 'County FIPS Code', 'County', 'Site Latitude',
       'Site Longitude'],
      dtype='object')


In [21]:
print("\n" + "="*50)
print("Column analysis for potential removal:")

print("\n1. Source column:")
print(f"Unique values: {ozone['Source'].unique()}")
print(f"Value counts: {ozone['Source'].value_counts()}")

print("\n3. POC column (Parameter Occurrence Code):")
print(f"Unique values: {ozone['POC'].unique()}")
print(f"Value counts: {ozone['POC'].value_counts()}")

print("\n2. Units column:")
print(f"Unique values: {ozone['Units'].unique()}")
print(f"Value counts: {ozone['Units'].value_counts()}")


Column analysis for potential removal:

1. Source column:
Unique values: ['AQS' 'AirNow']
Value counts: Source
AQS       48269
AirNow     6490
Name: count, dtype: int64

3. POC column (Parameter Occurrence Code):
Unique values: [1 2]
Value counts: POC
1    54745
2       14
Name: count, dtype: int64

2. Units column:
Unique values: ['ppm']
Value counts: Units
ppm    54759
Name: count, dtype: int64


### Remove Columns

In [24]:
reduced_df = ozone.drop(columns=['Units', 'CBSA Code', 'County FIPS Code'])
reduced_df.tail(3)

,Date,Source,Site ID,POC,Daily Max 8-hour Ozone Concentration,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,Method Code,CBSA Name,County,Site Latitude,Site Longitude
54756,June 30/2024,AQS,60690002,1,NaN,29.0,Hollister,17,100.0,47.0,"San Jose-Sunnyvale-Santa Clara, CA",San Benito,36.843425,-121.362100
54757,/2024,AQS,61072010,1,0.058,61.0,Porterville,17,100.0,199.0,"Visalia-Porterville, CA",Tulare,36.031831,-119.055018
54758,July 22/2024,AQS,60611004,1,NaN,45.0,Tahoe City-Fairway Drive,17,100.0,87.0,"Sacramento--Roseville--Arden-Arcade, CA",Placer,39.166017,-120.148833
